In [1]:
#Import libraries for simulation
import tensorflow as tf
import numpy as np

#Imports for visualization
import PIL.Image
from io import BytesIO
from IPython.display import clear_output, Image, display

In [2]:
### This notebook computes the gradient of a PDE scheme result
### This is useful to get the greeks in the case of option pricing

In [3]:
'''def DisplayArray(a, fmt='jpeg', rng=[0,1]):
  """Display an array as a picture."""
  a = (a - rng[0])/float(rng[1] - rng[0])*255
  a = np.uint8(np.clip(a, 0, 255))
  f = BytesIO()
  PIL.Image.fromarray(a).save(f, fmt)
  clear_output(wait = True)
  display(Image(data=f.getvalue()))'''

'def DisplayArray(a, fmt=\'jpeg\', rng=[0,1]):\n  """Display an array as a picture."""\n  a = (a - rng[0])/float(rng[1] - rng[0])*255\n  a = np.uint8(np.clip(a, 0, 255))\n  f = BytesIO()\n  PIL.Image.fromarray(a).save(f, fmt)\n  clear_output(wait = True)\n  display(Image(data=f.getvalue()))'

In [4]:
sess = tf.InteractiveSession()

In [5]:
def make_kernel(a):
  """Transform a 2D array into a convolution kernel"""
  a = np.asarray(a)
  a = a.reshape(list(a.shape) + [1,1])
  return tf.constant(a, dtype=1)

def simple_conv(x, k):
  """A simplified 2D convolution operation"""
  x = tf.expand_dims(tf.expand_dims(x, 0), -1)
  y = tf.nn.depthwise_conv2d(x, k, [1, 1, 1, 1], padding='SAME')
  return y[0, :, :, 0]

def laplace(x):
  """Compute the 2D laplacian of an array"""
  laplace_k = make_kernel([[0.5, 1.0, 0.5],
                           [1.0, -6., 1.0],
                           [0.5, 1.0, 0.5]])
  return simple_conv(x, laplace_k)

In [6]:
N = 100

# Initial Conditions -- some rain drops hit a pond

# Set everything to zero
u_init = np.zeros([N, N], dtype=np.float32)
ut_init = np.zeros([N, N], dtype=np.float32)

# Some rain drops hit a pond at random points
for n in range(40):
  a,b = np.random.randint(0, N, 2)
  u_init[a,b] = np.random.uniform()

#DisplayArray(u_init, rng=[-0.1, 0.1])

In [7]:
# Parameters:
# eps -- time resolution
# damping -- wave damping
eps = tf.placeholder(tf.float32, shape=())
damping = tf.placeholder(tf.float32, shape=())

'''
# Create variables for simulation state
U  = tf.Variable(u_init)
Ut = tf.Variable(ut_init)

# Discretized PDE update rules
U_ = U + eps * Ut
Ut_ = Ut + eps * (laplace(U) - damping * Ut)
'''

# Best to store each time step in a list to get the final gradient
# The group and re-assign method computes a 0 gradients (or maybe there is another way ?)
Us  = [tf.Variable(u_init)] 
Uts = [tf.Variable(ut_init)]

# Operation to update the state
'''step = tf.group(
  U.assign(U_),
  Ut.assign(Ut_))'''

'step = tf.group(\n  U.assign(U_),\n  Ut.assign(Ut_))'

In [8]:
# Initialize state to initial conditions
tf.global_variables_initializer().run()

In [9]:
'''
# Run 1000 steps of PDE
for i in range(100):
  # Step simulation
  step.run({eps: 0.03, damping: 0.04})
  #DisplayArray(U.eval(), rng=[-0.1, 0.1])
'''

'\n# Run 1000 steps of PDE\nfor i in range(100):\n  # Step simulation\n  step.run({eps: 0.03, damping: 0.04})\n  #DisplayArray(U.eval(), rng=[-0.1, 0.1])\n'

In [10]:
for i in range(1,100):
    Us.append( Us[i-1] + eps * Uts[i-1] )
    Uts.append( Uts[i-1] + eps * (laplace(Us[i-1]) - damping * Uts[i-1]) )

In [11]:
output = tf.trace(Us[9])

In [12]:
grads = tf.gradients(output, [eps, damping])

In [13]:
grads[1].eval({eps: 0.03, damping: 0.04})

0.0035143339